This is a slight variation of tutorial 1
to test camera issues between different versions of Carla
Version 9.14 had a problem on Windows where in non-standard
aspect ratios it was scrambling camera image.
This code is to test for these.
Makes sure you have mutiple version of Carla installed
so you can pick specific kernels for this notebook to run
different Carla versions.
Instructions:
1. Open v9.14 Sim and select v9.14 kernel
2. Run all steps
3. See if camera windows looks good
4. Do all of the above steps with v9.15

And do not forget to re-launch different Sim for that version

In [1]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping

In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)


In [3]:
#define environment/world and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [4]:
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

In [5]:
#spawn a car in a random location

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)


In [6]:
# move simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+3 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point)

In [7]:
#send the car off on autopilot - this will leave the spectator
vehicle.set_autopilot(True)

In [8]:
#setting RGB Camera - this follow the approach explained in a Carla video
# link: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s

#camera mount offset on the car - you can tweak these to each car to avoid any parts of the car being in the view
CAMERA_POS_Z = 1.6 #this means 1.6m up from the ground
CAMERA_POS_X = 0.9 #this is 0.9m forward

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '360') # 16:9 type ratios work in CARLA 9.14 on Windows so we change it to something different here
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

In [9]:
# this will open RGB camera window, select it from the task bar if it does not pop up
# you will see the result - if camera works properly
while True:
      
    # Dispaly with imshow
    cv2.imshow('All cameras',camera_data['image'])
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

-1

In [11]:
# clean up after yourself

camera.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

: 